# Exploring the NIFTY 50 index under different SMA regimes

Markets tend to be manic depressive, going from excessive optimism to excessive pessimism. The distribution of returns tend to be different based on the "regime." A simple way to demarcate these regimes is by using an SMA (Simple Moving Average) to bucket returns into binary groups. Here, we explore how returns are distributed in these buckets.

In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(RPostgres)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(reshape2)
library(ggrepel)
library(lubridate)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

#initialize
indices<-Indices()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 

Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘PerformanceAnalytics’

The following object is masked from

In [2]:
smaLb <- 50 #number of days
px <- indices$NseTimeSeries() %>%
    filter(NAME == 'NIFTY 50 TR') %>%
    select(TIME_STAMP, CLOSE) %>%
    collect()

numDays <- px %>%
    group_by(Y = year(TIME_STAMP)) %>%
    summarize(N = n())

numDays %>% print(n=Inf)

# A tibble: 21 x 2
       Y     N
   <dbl> <int>
 1  1999   129
 2  2000   250
 3  2001   248
 4  2002   251
 5  2003   254
 6  2004   254
 7  2005   251
 8  2006   250
 9  2007   249
10  2008   246
11  2009   243
12  2010   252
13  2011   247
14  2012   251
15  2013   250
16  2014   244
17  2015   248
18  2016   247
19  2017   248
20  2018   246
21  2019   143


In [3]:
pXts <- xts(px$CLOSE, px$TIME_STAMP)
names(pXts) <- c('INDEX')
pXts$SMA <- SMA(pXts[,1], smaLb)
pXts$D_RET <- dailyReturn(pXts$INDEX)

yrs <- unique(year(index(pXts)))

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)